In [1]:
# https://leetcode.com/problems/merge-intervals/

# Given an array of intervals where intervals[i] = [starti, endi], merge all overlapping intervals, and return an array of the non-overlapping intervals that cover all the intervals in the input.

 

# Example 1:

# Input: intervals = [[1,3],[2,6],[8,10],[15,18]]
# Output: [[1,6],[8,10],[15,18]]
# Explanation: Since intervals [1,3] and [2,6] overlaps, merge them into [1,6].
    
class Solution(object):
    def merge(self, intervals):
        if len(intervals) == 0:
            return []
        
        intervals = sorted(intervals, key=lambda k:k[0])
        
        out = [intervals[0]]
        for i in range(1, len(intervals)):
            if intervals[i][0] <= out[-1][1]:
                out[-1][1] = max(out[-1][1], intervals[i][1])
            else:
                out.append(intervals[i])
        
        return out

In [2]:
# https://leetcode.com/problems/maximum-sum-bst-in-binary-tree/

# Given a binary tree root, the task is to return the maximum sum of all keys of any sub-tree which is also a Binary Search Tree (BST).

# Assume a BST is defined as follows:

# The left subtree of a node contains only nodes with keys less than the node's key.
# The right subtree of a node contains only nodes with keys greater than the node's key.
# Both the left and right subtrees must also be binary search trees.
 

class Solution(object):
    def sub_tree_bst(self, root, cache):
        if root is None:
            return [-float("Inf"), float("Inf"), -float("Inf"), True]
        
        a = self.sub_tree_bst(root.left, cache)
        b = self.sub_tree_bst(root.right, cache)
        
        f = a[3] and b[3] and a[2] < root.val < b[1]
        
        sums = 0
        if a[0] != -float("Inf"):
            sums += a[0]
        
        if b[0] != -float("Inf"):
            sums += b[0]
        
        cache[root] = [sums+root.val, min(root.val, a[1], b[1]), max(root.val, a[2], b[2]), f]
        return cache[root]
        
    def maxSumBST(self, root):
        if root is None:
            return 0
        
        cache = {}
        self.sub_tree_bst(root, cache)
        
        max_sum = 0
        
        for x, y in cache.items():
            if y[3]:
                max_sum = max(max_sum, y[0])
                
        return max_sum

In [3]:
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-transaction-fee/

# You are given an array prices where prices[i] is the price of a given stock on the ith day, and an integer fee representing a transaction fee.

# Find the maximum profit you can achieve. You may complete as many transactions as you like, but you need to pay the transaction fee for each transaction.

# Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).

# Example 1:

# Input: prices = [1,3,2,8,4,9], fee = 2
# Output: 8
# Explanation: The maximum profit can be achieved by:
# - Buying at prices[0] = 1
# - Selling at prices[3] = 8
# - Buying at prices[4] = 4
# - Selling at prices[5] = 9
# The total profit is ((8 - 1) - 2) + ((9 - 4) - 2) = 8.

class Solution(object):
    def maxProfit(self, prices, fee):
        windows = []
        start = 0
        
        for i in range(1, len(prices)):
            if prices[i] < prices[i-1]:
                if i-1 > start:
                    windows.append((start, i-1))
                start = i
        
        if len(prices)-1 > start:
            windows.append((start, len(prices)-1))
            
        if len(windows) == 0:
            return 0
        
        curr_best_buy = 0
        profit = [0]*len(windows)
        
        profit[0] = max(profit[0], prices[windows[0][1]]-prices[windows[0][0]]-fee)
        
        for i in range(1, len(windows)):
            a = profit[i-1] + prices[windows[i][1]] - prices[windows[i][0]] - fee
            
            b = profit[curr_best_buy-1] + prices[windows[i][1]] - prices[windows[curr_best_buy][0]] - fee if curr_best_buy > 0 else prices[windows[i][1]] - prices[windows[curr_best_buy][0]] - fee
            
            profit[i] = max(profit[i-1], a, b)
            
            if a > b:
                curr_best_buy = i
        
        return profit[-1]
    
    
class Solution(object):
    def maxProfit(self, prices, fee):
        if len(prices) <= 1:
            return 0
        
        buy, sell = [0]*len(prices), [0]*len(prices)
        
        for i in range(len(prices)):
            if i == 0:
                buy[i] = -prices[0]
                sell[i] = 0
            else:
                buy[i] = max(buy[i-1], sell[i-1] - prices[i]) if i-2 >= 0 else max(buy[i-1], -prices[i])
                sell[i] = max(sell[i-1], buy[i-1] + prices[i] - fee) 
        
        return sell[-1]

In [ ]:
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/

# You are given an array prices where prices[i] is the price of a given stock on the ith day.

# Find the maximum profit you can achieve. You may complete as many transactions as you like (i.e., buy one and sell one share of the stock multiple times) with the following restrictions:

# After you sell your stock, you cannot buy stock on the next day (i.e., cooldown one day).
# Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).


# Example 1:

# Input: prices = [1,2,3,0,2]
# Output: 3
# Explanation: transactions = [buy, sell, cooldown, buy, sell]

class Solution(object):
    def maxProfit(self, prices):
        if len(prices) <= 1:
            return 0
        
        prev_small = [-1]*len(prices)
        stack = []
        
        for i in range(len(prices)):
            while len(stack) > 0 and prices[stack[-1]] >= prices[i]:
                stack.pop()
            
            if len(stack) > 0:
                prev_small[i] = stack[-1]
            
            stack.append(i)
        
        profit = [0]*len(prices)
        curr_best_buy = 0
        
        for i in range(len(prices)):
            j = prev_small[i]
            
            if j != -1:
                a = profit[j-2] + prices[i] - prices[j] if j > 1 else prices[i] - prices[j]
                b = profit[curr_best_buy-2] + prices[i] - prices[curr_best_buy] if curr_best_buy > 1 else prices[i] - prices[curr_best_buy]
                
                profit[i] = max(profit[i-1], a, b)
                
                if a > b:
                    curr_best_buy = j
            else:
                profit[i] = profit[i-1] if i > 0 else 0
        
        return profit[-1]
    
    
class Solution(object):
    def maxProfit(self, prices):
        if len(prices) <= 1:
            return 0
        
        buy, sell = [0]*len(prices), [0]*len(prices)
        
        for i in range(len(prices)):
            if i == 0:
                buy[i] = -prices[0]
                sell[i] = 0
            else:
                buy[i] = max(buy[i-1], sell[i-2] - prices[i]) if i-2 >= 0 else max(buy[i-1], -prices[i])
                sell[i] = max(sell[i-1], buy[i-1] + prices[i]) 
        
        return sell[-1]

In [4]:
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/

# You are given an array prices where prices[i] is the price of a given stock on the ith day.

# Find the maximum profit you can achieve. You may complete at most two transactions.

# Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).

# Example 1:

# Input: prices = [3,3,5,0,0,3,1,4]
# Output: 6
# Explanation: Buy on day 4 (price = 0) and sell on day 6 (price = 3), profit = 3-0 = 3.
# Then buy on day 7 (price = 1) and sell on day 8 (price = 4), profit = 4-1 = 3.

class Solution(object):
    def maxProfit(self, prices):
        min_price = prices[0]
        max_p = [0]*len(prices)
        
        for i in range(1, len(prices)):
            p = prices[i]-min_price
            max_p[i] = max(max_p[i-1], p) if i > 0 else p
            min_price = min(min_price, prices[i])
        
        max_price = prices[-1]
        out = 0
        
        for i in range(len(prices)-2, -1, -1):
            p = max_price-prices[i]
            out = max(out, p, max_p[i], p+max_p[i-1]) if i > 0 else max(out, p, max_p[i])
            max_price = max(max_price, prices[i])
        
        return out

In [5]:
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iv/

# You are given an integer array prices where prices[i] is the price of a given stock on the ith day, and an integer k.

# Find the maximum profit you can achieve. You may complete at most k transactions.

# Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).

# Example 1:

# Input: k = 2, prices = [2,4,1]
# Output: 2
# Explanation: Buy on day 1 (price = 2) and sell on day 2 (price = 4), profit = 4-2 = 2.

class Solution(object):
    def maxProfit(self, k, prices):
        windows = []
        start = 0
        
        for i in range(1, len(prices)):
            if prices[i] < prices[i-1]:
                if i-1 > start:
                    windows.append((start, i-1))
                start = i
        
        if len(prices)-1 > start:
            windows.append((start, len(prices)-1))
            
        if len(windows) == 0:
            return 0
        
        dp = [[0]*len(windows) for i in range(k+1)]
        
        for h in range(1, k+1):
            dp[h][0] = max(dp[h][0], prices[windows[0][1]]-prices[windows[0][0]])
            curr_best_buy = 0

            for i in range(1, len(windows)):
                a = dp[h-1][i-1] + prices[windows[i][1]] - prices[windows[i][0]] if h > 1 else prices[windows[i][1]] - prices[windows[i][0]]

                if h > 1:
                    b = dp[h-1][curr_best_buy-1] + prices[windows[i][1]] - prices[windows[curr_best_buy][0]] if curr_best_buy > 0 else prices[windows[i][1]] - prices[windows[curr_best_buy][0]]
                else:
                    b = prices[windows[i][1]] - prices[windows[curr_best_buy][0]]

                dp[h][i] = max(dp[h][i-1], a, b)

                if a > b:
                    curr_best_buy = i
        
        return dp[k][len(windows)-1]

In [9]:
# https://leetcode.com/discuss/interview-question/124734/Facebook-or-Phone-Screen-or-Tuple-Combination-Sum

# Q: Write a function that list any 3 elements that sum to zero, from an array of integers.

# Notes:

# We need all unique solutions. (-5, -5, 10) is same as (10, -5, -5)
# The same element can be used multiple times.
# Example
# Input: [-5, 1, 10, 2, 3] // array of integers
# Output: [(-5, -5, 10), (-5, 2, 3)] // array of tuples
    
    
def tuple_combination_sum(arr):
    sums = {}
    out = []
    
    for i in range(len(arr)-1, -1, -1):
        for j in range(i, len(arr)):
            s = arr[i]+arr[j]
            if s not in sums:
                sums[s] = []
            sums[s].append((arr[i], arr[j]))
        
        if -arr[i] in sums:
            for x, y in sums[-arr[i]]:
                out.append((arr[i], x, y))
        
    return out

In [10]:
tuple_combination_sum([-5, 1, 10, 2, 3])

[(-5, 2, 3), (-5, -5, 10)]

In [35]:
# https://leetcode.com/discuss/interview-question/125179/Largest-crossword-matrix

# Given a large matrix of letters and a large dictionary of words, return the largest area matrix from (0, 0) to (i, j) where every row and column spell a valid word.

# For example, given the matrix below,

# [ ["R", "I", "B", "B"],
#   ["Y", "O", "U", "C"],
#   ["E", "N", "D", "A"],
#   ["T", "H", "E", "Y"]
# ]
# The resulting matrix is below because every row and column spell out a word ("RIB", "YOU", "END", "RYE", "ION", "BUD").

# [ ["R", "I", "B"],
#   ["Y", "O", "U"],
#   ["E", "N", "D"]
# ]

def largest_crossword_matrix(mat, words):
    words = set(words)
    
    row = [[0]*len(mat[0]) for i in range(len(mat))]
    col = [[0]*len(mat[0]) for i in range(len(mat))]
    
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            p = ''
            row[i][j] = -1
            
            for k in range(j, len(mat[0])):
                p += mat[i][k]
                if p in words:
                    row[i][j] = k-j+1
    
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            p = ''
            col[i][j] = -1
            
            for k in range(i, len(mat)):
                p += mat[k][j]
                if p in words:
                    col[i][j] = k-i+1
                    
    max_sq = 0
    s = None
    
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            a = row[i][j]
            b = col[i][j]
            
            if a != -1 and b != -1:
                c = min(a, b)
                flag = True
                
                for k in range(j+1, j+c):
                    if col[i][k] < c:
                        flag = False
                        break
                
                if flag:
                    for k in range(i+1, i+c):
                        if row[k][j] < c:
                            flag = False
                            break
                
                if flag:
                    if c > max_sq:
                        max_sq = c
                        s = (i, j)
    
    out = [[0]*max_sq for i in range(max_sq)]
    
    for i in range(s[0], s[0]+c):
        for j in range(s[1], s[1]+c):
            out[i-s[0]][j-s[1]] = mat[i][j]
            
    return out

largest_crossword_matrix([["R", "I", "B", "B"],
                          ["Y", "O", "U", "C"],
                          ["E", "N", "D", "A"],
                          ["T", "H", "E", "Y"]], ["RIB", "YOU", "END", "RYE", "ION", "BUD"])

[['R', 'I', 'B'], ['Y', 'O', 'U'], ['E', 'N', 'D']]

In [36]:
# https://leetcode.com/problems/minimum-add-to-make-parentheses-valid/submissions/

# Given a string s of '(' and ')' parentheses, we add the minimum number of parentheses ( '(' or ')', and in any positions ) so that the resulting parentheses string is valid.

# Formally, a parentheses string is valid if and only if:

# It is the empty string, or
# It can be written as AB (A concatenated with B), where A and B are valid strings, or
# It can be written as (A), where A is a valid string.
# Given a parentheses string, return the minimum number of parentheses we must add to make the resulting string valid.

 

# Example 1:

# Input: s = "())"
# Output: 1

class Solution(object):
    def minAddToMakeValid(self, s):
        if len(s) == 0:
            return 0
        
        stack = []
        
        for i in range(len(s)):
            if s[i] == ')':
                if len(stack) > 0 and s[stack[-1]] == '(':
                    stack.pop()
                else:
                    stack.append(i)
            else:
                stack.append(i)
        
        return len(stack)

In [37]:
# https://leetcode.com/problems/video-stitching/

# You are given a series of video clips from a sporting event that lasted time seconds. These video clips can be overlapping with each other and have varying lengths.

# Each video clip is described by an array clips where clips[i] = [starti, endi] indicates that the ith clip started at starti and ended at endi.

# We can cut these clips into segments freely.

# For example, a clip [0, 7] can be cut into segments [0, 1] + [1, 3] + [3, 7].
# Return the minimum number of clips needed so that we can cut the clips into segments that cover the entire sporting event [0, time]. If the task is impossible, return -1.

 

# Example 1:

# Input: clips = [[0,2],[4,6],[8,10],[1,9],[1,5],[5,9]], time = 10
# Output: 3
# Explanation: 
# We take the clips [0,2], [8,10], [1,9]; a total of 3 clips.
# Then, we can reconstruct the sporting event as follows:
# We cut [1,9] into segments [1,2] + [2,8] + [8,9].
# Now we have segments [0,2] + [2,8] + [8,10] which cover the sporting event [0, 10].

class Solution(object):
    def videoStitching(self, clips, T):
        if T == 0:
            return 0
        
        clips = sorted(clips, key=lambda k:(k[0], -k[1]))
        
        if clips[0][0] > 0:
            return -1
        
        if clips[0][1] >= T:
            return 1
        
        stack = [clips[0]]
        
        for i in range(1, len(clips)):
            x = None
            
            while len(stack) > 0 and clips[i][0] <= stack[-1][1] and clips[i][1] > stack[-1][1]:
                x = stack.pop()
            
            if x is not None:
                stack.append(x)
                stack.append(clips[i])
                
            if stack[-1][1] >= T:
                return len(stack)
                
        return -1

In [50]:
# https://leetcode.com/discuss/interview-question/124825/Facebook-or-Least-number-of-intervals-to-cover-target-interval

# Given a list of intervals A and one interval B, find the least number of intervals from A that can fully cover B.
# If cannot find any result, just return 0.

# Example 1:

# Input: A = [[0, 3], [3, 4], [4, 6], [2, 7]] B = [0,6]
# Output: 2
# Explanation: We can use [0, 3] [2, 7] to cover the B
    
def cover_target_interval(intervals, target):
    intervals = [intervals[i] for i in range(len(intervals)) if intervals[i][1] >= target[0] and intervals[i][0] <= target[1]]
    intervals = sorted(intervals, key=lambda k:(k[0], -k[1]))
    
    stack = [intervals[0]]
        
    for i in range(1, len(intervals)):
        x = None

        while len(stack) > 0 and intervals[i][0] <= stack[-1][1] and intervals[i][1] > stack[-1][1]:
            x = stack.pop()

        if x is not None:
            stack.append(x)
            stack.append(intervals[i])

        if stack[-1][1] >= target[1]:
            return len(stack)

    return 0

In [54]:
# https://leetcode.com/discuss/interview-question/124616/Facebook-or-Merge-two-interval-lists

# Given A and B two interval lists, A has no overlap inside A and B has no overlap inside B. Write the function to merge two interval lists, output the result with no overlap. Ask for a very efficient solution

# A naive method can combine the two list, and sort and apply merge interval in the leetcode, but is not efficient enough.

# For example,
# A: [1,5], [10,14], [16,18]
# B: [2,6], [8,10], [11,20]

# output [1,6], [8, 20]

def merge_interval_list(a, b):
    a = sorted(a, key=lambda k:k[0])
    b = sorted(b, key=lambda k:k[0])
    
    out = []
    i, j = 0, 0
    
    while i < len(a) and j < len(b):
        if a[i][0] <= b[j][0]:
            if len(out) == 0:
                out.append(a[i])
            else:
                if a[i][0] <= out[-1][1]:
                    out[-1][1] = max(out[-1][1], a[i][1])
                else:
                    out.append(a[i])
        
        else:
            if len(out) == 0:
                out.append(b[j])
            else:
                if b[j][0] <= out[-1][1]:
                    out[-1][1] = max(out[-1][1], b[j][1])
                else:
                    out.append(b[j])
        
        if a[i][1] < b[j][1]:
            i += 1
        elif a[i][1] > b[j][1]:
            j += 1
        else:
            i += 1
            j += 1
    
    while i < len(a):
        if a[i][0] <= out[-1][1]:
            out[-1][1] = max(out[-1][1], a[i][1])
        else:
            out.append(a[i])
            
        i += 1
    
    while j < len(b):
        if b[j][0] <= out[-1][1]:
            out[-1][1] = max(out[-1][1], b[j][1])
        else:
            out.append(b[j])
            
        j += 1
            
    return out     

In [31]:
# https://leetcode.com/discuss/interview-question/124708/Print-JSON-format-String

# This was a recent interview question. Suppose we have given following string:

# "{"id": "0001", "type": "donut","name": "Cake","ppu": 0.55, "batters":{"batter":[{ "id": "1001", "type": "Regular" },{ "id": "1002", "type": "Chocolate" }]},"topping":[{ "id": "5001", "type": "None" },{ "id": "5002", "type": "Glazed" }]}"
# we need to print it in following order:

# {
# 	"id": "0001",
# 	"type": "donut",
# 	"name": "Cake",
# 	"ppu": 0.55,
# 	"batters":
# 		{
# 			"batter":
# 				[
# 					{ "id": "1001", "type": "Regular" },
# 					{ "id": "1002", "type": "Chocolate" }
# 				]
# 		},
# 	"topping":
# 		[
# 			{ "id": "5001", "type": "None" },
# 			{ "id": "5002", "type": "Glazed" }
# 		]
# }


def json_print(s):
    stack = []
    ident = 0
    p = ''
    
    for i in range(len(s)):
        if s[i] == ',':
            p += s[i]
            if len(p) > 0:
                stack.append((p, ident))
            p = ''
        
        elif s[i] == '{':
            if len(p) > 0:
                stack.append((p, ident))
            stack.append(('{', ident))
            p = ''
            ident += 4
            
        elif s[i] == '[':
            if len(p) > 0:
                stack.append((p, ident))
            stack.append(('[', ident))
            p = ''
            ident += 4
            
        elif s[i] == '}':
            ident -= 4
            stack.append(('}', ident))
            p = ''
            
        elif s[i] == ']':
            ident -= 4
            stack.append((']', ident))
            p = ''
            
        else:
            p += s[i]
    
    out = ''
    for x, y in stack:
        out += ' '*y + x.strip() + '\r\n'
    
    return out

print(json_print('{"id": "0001", "type": "donut", "name": "Cake", "ppu": 0.55, "batters":{"batter":[{ "id": "1001", "type": "Regular" },{ "id": "1002", "type": "Chocolate" }]},"topping":[{ "id": "5001", "type": "None" },{ "id": "5002", "type": "Glazed" }]}'))

{
    "id": "0001",
    "type": "donut",
    "name": "Cake",
    "ppu": 0.55,
    "batters":
    {
        "batter":
        [
            {
                "id": "1001",
            }
            ,
            {
                "id": "1002",
            }
        ]
    }
    ,
    "topping":
    [
        {
            "id": "5001",
        }
        ,
        {
            "id": "5002",
        }
    ]
}



In [32]:
# https://www.lintcode.com/problem/907/

# Given a non-empty string s and an integer k, rearrange the string such that the same characters are at least distance k from each other.

# All input strings are given in lowercase letters. If it is not possible to rearrange the string, return an empty string "".

# Example
# Example 1:

# Input: s = "aabbcc", k = 3
# Output: "abcabc"
# Explanation:
# The same letters are at least distance 3 from each other.

class Solution:
    """
    @param s: a string
    @param k: an integer
    @return: a string such that the same characters are at least distance k from each other
    """
    def rearrangeString(self, s, k):
        char_cnts = {}

        for x in s:
            if x not in char_cnts:
                char_cnts[x] = 0
            char_cnts[x] += 1
        
        heap = [(-char_cnts[x], x) for x in char_cnts]
        heapq.heapify(heap)
        prev_pos = {}
        out = ['']*len(s)
        i = 0

        while len(heap) > 0:
            w, x = heapq.heappop(heap)

            while i < len(s) and out[i] != '':
                i += 1
            
            if i == len(s):
                return ''

            if x not in prev_pos or i-prev_pos[x] >= k:
                out[i] = x
                prev_pos[x] = i
                i += 1

            elif prev_pos[x]+k < len(s):
                out[prev_pos[x]+k] = x
                prev_pos[x] += k
            
            else:
                return ''
            
            w = -w

            if w > 1:
                heapq.heappush(heap, (-(w-1), x))
        
        return ''.join(out)

In [59]:
# https://www.geeksforgeeks.org/find-the-point-where-maximum-intervals-overlap/

# Consider a big party where a log register for guest’s entry and exit times is maintained. Find the time at which there are maximum guests in the party. Note that entries in register are not in any order.
# Example : 

# Input: arrl[] = {1, 2, 9, 5, 5}
#        exit[] = {4, 5, 12, 9, 12}
# First guest in array arrives at 1 and leaves at 4, 
# second guest arrives at 2 and leaves at 5, and so on.

# Output: 5
# There are maximum 3 guests at time 5. 

def maximum_overlap_point(intervals):
    starts = [(x, 1) for x , y in intervals]
    ends = [(y, -1) for x, y in intervals]
    
    sorted_intervals = sorted(starts+ends, key=lambda k:(k[0], -k[1]))
    
    num_intervals = 0
    intersections = {}
    max_intersect = 0
    
    for i in range(len(sorted_intervals)):
        max_intersect = max(max_intersect, num_intervals)
        intersections[sorted_intervals[i][0]] = max(intersections[sorted_intervals[i][0]], num_intervals) if sorted_intervals[i][0] in intersections else num_intervals
        
        if sorted_intervals[i][1] == 1:
            num_intervals += 1
        else:
            num_intervals -= 1
    
    for x, y in intersections.items():
        if y == max_intersect:
            return x
    
    return -1

maximum_overlap_point([[1,4],[2,5],[9,12],[5,9],[5,12]])

In [61]:
# https://leetcode.com/discuss/interview-question/124723/Facebook-or-Phone-Screen-or-Amazing-Number

# Define amazing number as: its value is less than or equal to its index. Given a circular array, find the starting position, such that the total number of amazing numbers in the array is maximized.
# Example 1: 0, 1, 2, 3
# Ouptut: 0. When starting point at position 0, all the elements in the array are equal to its index. So all the numbers are amazing number.
# Example 2: 1, 0 , 0
# Output: 1. When starting point at position 1, the array becomes 0, 0, 1. All the elements are amazing number.
# If there are multiple positions, return the smallest one.

def amazing_number(arr):
    intervals = []
    
    for i in range(len(arr)):
        if arr[i]-i < 0:
            h = len(arr)-i+arr[i]
            g = len(arr)-1
            intervals.append([h, g])
            intervals.append([0, len(arr)-1-i])
        else:
            intervals.append([arr[i]-i, len(arr)-1-i])
        
    return maximum_overlap_point(intervals)

amazing_number([1,0,0])

In [55]:
# https://leetcode.com/problems/decode-ways/

# A message containing letters from A-Z can be encoded into numbers using the following mapping:

# 'A' -> "1"
# 'B' -> "2"
# ...
# 'Z' -> "26"
# To decode an encoded message, all the digits must be grouped then mapped back into letters using the reverse of the mapping above (there may be multiple ways). For example, "11106" can be mapped into:

# "AAJF" with the grouping (1 1 10 6)
# "KJF" with the grouping (11 10 6)
# Note that the grouping (1 11 06) is invalid because "06" cannot be mapped into 'F' since "6" is different from "06".

# Given a string s containing only digits, return the number of ways to decode it.

# The answer is guaranteed to fit in a 32-bit integer.

 

# Example 1:

# Input: s = "12"
# Output: 2
# Explanation: "12" could be decoded as "AB" (1 2) or "L" (12).

class Solution(object):
    def num_decode(self, s, i, cache):
        if i == len(s):
            return 1
        
        if s[i] == "0":
            return 0
        
        p = ''
        n = 0
        for j in range(i, len(s)):
            p += s[j]
            if 1 <= int(p) <= 26:
                a = self.num_decode(s, j+1, cache) if j+1 not in cache else cache[j+1]
                n += a
            else:
                break
        
        cache[i] = n
        return n
            
    def numDecodings(self, s):
        return self.num_decode(s, 0, {})

In [71]:
# https://leetcode.com/discuss/interview-question/124759/Facebook-or-Onsite-or-Randomly-generate-mines-on-a-grid

# You are given a m*n grid. You are asked to generate k mines on this grid randomly. Each cell should have equal probability of k / m*n of being chosen.

import random

def randomly_generate_mines(k, m, n):
    curr = list(range(k))
    
    for i in range(k, m*n):
        j = random.randint(0, i)
        if j < k:
            curr[j] = i
    
    mines = [[0]*n for i in range(m)]
    
    for x in curr:
        r = int(x/n)
        c = x % n
        mines[r][c] = 1
    
    return mines

print(randomly_generate_mines(5, 5, 5))

[[1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [72]:
# https://leetcode.com/discuss/interview-question/240104/Facebook-or-Onsite-or-Number-of-squares

# Input: A list of vertical and horizontal lines with its start point and end point in a graph.
# Output: Find the total number of squares formed up by all these lines.
    
def count_squares(start_points, end_points):
    horizontal_lines = [i for i in range(len(start_points)) if start_points[i][1] == end_points[i][1]]
    vertical_lines = [i for i in range(len(start_points)) if start_points[i][0] == end_points[i][0]]
    
    intersection = []
    
    for i in range(len(horizontal_lines)):
        h_start = start_points[horizontal_lines[i]]
        h_end = end_points[horizontal_lines[i]]
        
        for j in range(len(vertical_lines)):
            v_start = start_points[vertical_lines[j]]
            v_end = end_points[vertical_lines[j]]
            
            if h_start[0] <= v_start[0] <= h_end[0] and v_start[1] <= h_start[1] <= v_end[1]:
                intersection += [(v_start[0], h_start[1])]
    
    intersection = set(intersection)
    cnts = 0
    
    for a, b in intersection:
        for c, d in intersection:
            if c > a and d > b and c-a == d-b:
                if (a, d) in intersection and (c, b) in intersection:
                    cnts += 1
                    
    return cnts

In [84]:
# https://leetcode.com/discuss/interview-question/133408/Facebook-or-Onsite-or-Generate-patterns

# Given a string as input, return the list of all the patterns possible:

# '1' : ['A', 'B', 'C'], 
# '2' : ['D', 'E'],
# '12' : ['X']
# '3' : ['P', 'Q']
# Example if input is '123', then output should be 
# [ADP, ADQ, AEP, AEQ, BDP, BDQ, BEP, BEQ, CDP, CDQ, CEP, CEQ, XP, XQ]

def generate(pat_map, inp, i, cache):
    if i == len(inp):
        return ['']
    
    p = ''
    out = []
    
    for j in range(i, len(inp)):
        p += inp[j]
        if p in pat_map:
            a = generate(pat_map, inp, j+1, cache) if j+1 not in cache else cache[j+1]
            if len(a) > 0:
                for x in a:
                    for y in pat_map[p]:
                        out.append(y + x)
                
    cache[i] = out
    return out

def generate_patterns(pat_map, inp):
    return generate(pat_map, inp, 0, {})

pat_map = {'1' : ['A', 'B', 'C'], 
'2' : ['D', 'E'],
'12' : ['X'],
'3' : ['P', 'Q']}

generate_patterns(pat_map, '123')

['ADP',
 'BDP',
 'CDP',
 'AEP',
 'BEP',
 'CEP',
 'ADQ',
 'BDQ',
 'CDQ',
 'AEQ',
 'BEQ',
 'CEQ',
 'XP',
 'XQ']

In [85]:
# https://leetcode.com/problems/exclusive-time-of-functions/

# On a single-threaded CPU, we execute a program containing n functions. Each function has a unique ID between 0 and n-1.

# Function calls are stored in a call stack: when a function call starts, its ID is pushed onto the stack, and when a function call ends, its ID is popped off the stack. The function whose ID is at the top of the stack is the current function being executed. Each time a function starts or ends, we write a log with the ID, whether it started or ended, and the timestamp.

# You are given a list logs, where logs[i] represents the ith log message formatted as a string "{function_id}:{"start" | "end"}:{timestamp}". For example, "0:start:3" means a function call with function ID 0 started at the beginning of timestamp 3, and "1:end:2" means a function call with function ID 1 ended at the end of timestamp 2. Note that a function can be called multiple times, possibly recursively.

# A function's exclusive time is the sum of execution times for all function calls in the program. For example, if a function is called twice, one call executing for 2 time units and another call executing for 1 time unit, the exclusive time is 2 + 1 = 3.

# Return the exclusive time of each function in an array, where the value at the ith index represents the exclusive time for the function with ID i.

class Solution(object):
    def exclusiveTime(self, n, logs):
        intervals = []
        
        for s in logs:
            a = s.split(':')
            if a[1] == 'start':
                intervals.append([int(a[0]), 0, int(a[2])])
            else:
                intervals.append([int(a[0]), 1, int(a[2])])
        
        out = [0]*n
        stack = [intervals[0]]
        
        for i in range(1, len(intervals)):
            if intervals[i][1] == 0:
                if len(stack) > 0:
                    if intervals[i-1][1] == 0:
                        out[stack[-1][0]] += intervals[i][2]-intervals[i-1][2]
                    else:
                        out[stack[-1][0]] += max(0, intervals[i][2]-intervals[i-1][2]-1)
                
                stack.append(intervals[i])
                
            else:
                if intervals[i-1][1] == 0:
                    out[stack[-1][0]] += intervals[i][2]-intervals[i-1][2]+1
                else:
                    out[stack[-1][0]] += intervals[i][2]-intervals[i-1][2]
                    
                stack.pop()
        
        return out

In [86]:
# https://leetcode.com/problems/interval-list-intersections/

# You are given two lists of closed intervals, firstList and secondList, where firstList[i] = [starti, endi] and secondList[j] = [startj, endj]. Each list of intervals is pairwise disjoint and in sorted order.

# Return the intersection of these two interval lists.

# A closed interval [a, b] (with a < b) denotes the set of real numbers x with a <= x <= b.

# The intersection of two closed intervals is a set of real numbers that are either empty or represented as a closed interval. For example, the intersection of [1, 3] and [2, 4] is [2, 3].

 

# Example 1:


# Input: firstList = [[0,2],[5,10],[13,23],[24,25]], secondList = [[1,5],[8,12],[15,24],[25,26]]
# Output: [[1,2],[5,5],[8,10],[15,23],[24,24],[25,25]]

class Solution(object):
    def intervalIntersection(self, A, B):
        i, j = 0, 0
        out = []

        while i < len(A) and j < len(B):
            if A[i][1] < B[j][1]:
                if A[i][1] >= B[j][0]:
                    x = max(A[i][0], B[j][0])
                    y = A[i][1]
                    out.append([x, y])
                i += 1

            elif A[i][1] > B[j][1]:
                if B[j][1] >= A[i][0]:
                    x = max(A[i][0], B[j][0])
                    y = B[j][1]
                    out.append([x, y])
                j += 1

            else:
                x = max(A[i][0], B[j][0])
                y = B[j][1]
                out.append([x, y])
                i += 1
                j += 1

        return out

In [2]:
# https://leetcode.com/discuss/interview-question/124671/Facebook-or-Phone-Screen-or-Cartesian-tree


class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        
def cartesian_tree(arr):
    if len(arr) == 0:
        return None
    
    x = min(arr)
    left_arr, right_arr = [], []
    
    root = None
    
    for i in range(len(arr)):
        if arr[i] == x:
            root = TreeNode(arr[i])
        else:
            if root is None:
                left_arr.append(arr[i])
            else:
                right_arr.append(arr[i])
    
    root.left = cartesian_tree(left_arr)
    root.right = cartesian_tree(right_arr)
    
    return root

def inorder_print(root):
    if root is None:
        return []
    
    a = inorder_print(root.left)
    b = inorder_print(root.right)
    
    return a + [root.val] + b

root = cartesian_tree([1,2,3,5])
inorder_print(root)

In [1]:
# https://leetcode.com/problems/range-sum-query-2d-immutable/

# Given a 2D matrix matrix, handle multiple queries of the following type:

# Calculate the sum of the elements of matrix inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).
# Implement the NumMatrix class:

# NumMatrix(int[][] matrix) Initializes the object with the integer matrix matrix.
# int sumRegion(int row1, int col1, int row2, int col2) Returns the sum of the elements of matrix inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).

class NumMatrix(object):
    def __init__(self, matrix):
        self.mat_sum = [[0]*len(matrix[0]) for i in range(len(matrix))]
        
        for i in range(len(matrix)):
            curr_row_sum = 0
            for j in range(len(matrix[0])):
                curr_row_sum += matrix[i][j]
                if i == 0:
                    self.mat_sum[i][j] = curr_row_sum
                else:
                    self.mat_sum[i][j] = self.mat_sum[i-1][j] + curr_row_sum

    def sumRegion(self, row1, col1, row2, col2):
        a = self.mat_sum[row2][col2]
        b = self.mat_sum[row1-1][col2] if row1 > 0 else 0
        c = self.mat_sum[row2][col1-1] if col1 > 0 else 0
        d = self.mat_sum[row1-1][col1-1] if row1 > 0 and col1 > 0 else 0
        return a - b - c + d

In [9]:
# https://leetcode.com/discuss/interview-question/275785/Facebook-or-Phone-screen-or-Count-subsets

# question:


# Input:

# Given an array A of
# -positive
# -sorted
# -no duplicate
# -integer

# A positive integer k


# Output:

# Count of all such subsets of A,
# Such that for any such subset S,
# Min(S) + Max(S) = k
# subset should contain atleast two elements

def count_subsets(arr, k):
    pos_map = {}
    cnt = 0
    
    for i in range(len(arr)-1, -1, -1):
        if k-arr[i] in pos_map:
            n = pos_map[k-arr[i]]-1-i
            cnt += 1<<n
        
        pos_map[arr[i]] = i
    
    return cnt

count_subsets([1,2,3,4,5], 5)

5

5